# Imports

In [11]:
from IPython.display import Image
from IPython.core.display import HTML

In [272]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Configs

In [12]:
IMG_URL = 'https://i.ibb.co/47bRcVy/bow-normalized.jpg'

# Ex. 1

In [14]:
Image(url=IMG_URL,
     width=500, height=500)

In [158]:
texts = ['я и ты', 'ты и я', 'я я и только я', 'только не я', 'он'] # сразу удалил для удобства мешающие знаки

In [159]:
vect_count = CountVectorizer(token_pattern=r'(?u)\b\w+\b') # меняем патерн выделения токенов по-умолчанию

In [160]:
texts_df = vect_count.fit_transform(texts)

In [161]:
df = pd.DataFrame(texts_df.toarray(), columns=vect_count.get_feature_names(), index=texts)

In [162]:
df

,и,не,он,только,ты,я
я и ты,1,0,0,0,1,1
ты и я,1,0,0,0,1,1
я я и только я,1,0,0,1,0,3
только не я,0,1,0,1,0,1
он,0,0,1,0,0,0


In [163]:
number_of_documents = df.shape[0] # общее кол-во документов

In [164]:
# считаем кол-во вхождений признака
idf_count = {}
for col in df.columns:
    idf_count[col] = np.sum(df[col] >= 1) # кол-во схождений признака в корпус

In [165]:
idf_count

{'и': 3, 'не': 1, 'он': 1, 'только': 2, 'ты': 2, 'я': 4}

In [166]:
for k in idf_count.keys():
    idf_count[k] = np.log((1+number_of_documents)/(1+idf_count[k])) + 1

In [167]:
idf_count

{'и': 1.4054651081081644,
 'не': 2.09861228866811,
 'он': 2.09861228866811,
 'только': 1.6931471805599454,
 'ты': 1.6931471805599454,
 'я': 1.1823215567939547}

In [168]:
df = df.apply(lambda x: x / x.sum(), axis=1)

In [169]:
df

,и,не,он,только,ты,я
я и ты,0.333333,0.000000,0.0,0.000000,0.333333,0.333333
ты и я,0.333333,0.000000,0.0,0.000000,0.333333,0.333333
я я и только я,0.200000,0.000000,0.0,0.200000,0.000000,0.600000
только не я,0.000000,0.333333,0.0,0.333333,0.000000,0.333333
он,0.000000,0.000000,1.0,0.000000,0.000000,0.000000


In [170]:
for col in df.columns:
    df[col] = df[col] * idf_count[col]

In [171]:
df

,и,не,он,только,ты,я
я и ты,0.468488,0.000000,0.000000,0.000000,0.564382,0.394107
ты и я,0.468488,0.000000,0.000000,0.000000,0.564382,0.394107
я я и только я,0.281093,0.000000,0.000000,0.338629,0.000000,0.709393
только не я,0.000000,0.699537,0.000000,0.564382,0.000000,0.394107
он,0.000000,0.000000,2.098612,0.000000,0.000000,0.000000


# Ex. 2

In [184]:
df = pd.read_csv('labeled.csv')

In [185]:
df.head(2)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0


In [186]:
target = df['toxic']
del df['toxic']

In [187]:
target[:4]

0    1.0
1    1.0
2    1.0
3    1.0
Name: toxic, dtype: float64

In [189]:
df.head(2)

,comment
0,"Верблюдов-то за что? Дебилы, бл...\n"
1,"Хохлы, это отдушина затюканого россиянина, мол..."


In [193]:
# векторизуем тексты
vect_idf = TfidfVectorizer()
texts = vect_idf.fit_transform(df['comment']) 

In [198]:
cosine_similarity(texts[3], texts[12666])

array([[0.27330886]])

In [240]:
# не нолевое сходство
np.sort(cosine_distances(texts[43], texts))[0, 1:4]

array([0.83956018, 0.88210851, 0.88590919])

In [241]:
most_similary_texts = cosine_distances(texts[43], texts).argsort()[0,1:4]

In [242]:
for i, text in enumerate(df.loc[most_similary_texts]['comment']):
    print(f'Индекс текста: {most_similary_texts[i]}')
    print('Текст:')
    print(text)
    print()

Индекс текста: 1986
Текст:
НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?


Индекс текста: 1957
Текст:
Че за бригада и че за махоун? Из полицейской академии?

Индекс текста: 6259
Текст:
Герка ебет только даунов которые игрли а него. Ибо и геймплей и сюжетто кусок говна.




In [243]:
# Оригинальный текст
df.loc[43]

comment    Люди зажрались и охуели если по мнению этих иг...
Name: 43, dtype: object

# Ex. 3 

In [267]:
text_for_predict = open('data/2ch_corpus.txt').readlines()

In [268]:
text_for_predict[:2]

[" Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество 

In [293]:
import string
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [299]:
def preprocessing(text):
    text = text.lower()
    
    remove = string.punctuation
    remove += '«»—…#№“”'
    
    text = ' '.join([ morph.parse((word.strip(remove)))[0].normal_form for word in text.split(' ')])
    
    return text

## 1 подход

In [300]:
df['comment'] = df['comment'].apply(preprocessing)

In [301]:
df.head(4)

,comment
0,верблюд-то за что дебил бл...\n
1,хохол это отдушина затюканый россиянин мол вон...
2,собака собачий смерть\n
3,страница обновить дебил это тоже не оскорблени...


In [302]:
# обучаемся и тестируемся на текстах из labeled.csv, предсказываем для 2ch_corpus.txt

In [303]:
vect_tfidf = TfidfVectorizer(
    ngram_range=(1,2),
    min_df = 5,
    max_df = 0.9,
    max_features = 2000,
    token_pattern=r'(?u)\b\w+\b'
)

In [304]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, train_size=0.2, stratify=target)

In [305]:
X_train = vect_tfidf.fit_transform(X_train)
X_test = vect_tfidf.transform(X_test)

In [311]:
clf_logreg = LogisticRegression(penalty='l2', n_jobs=-1, verbose=1, max_iter=1000)

In [312]:
clf_logreg.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.3s finished


LogisticRegression(max_iter=1000, n_jobs=-1, verbose=1)

In [313]:
y_pred = clf_logreg.predict(X_test)

In [314]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.96      0.86      7669
         1.0       0.86      0.48      0.61      3861

    accuracy                           0.80     11530
   macro avg       0.82      0.72      0.74     11530
weighted avg       0.81      0.80      0.78     11530



In [315]:
# вроде норм качество

In [316]:
text_for_predict = [preprocessing(text) for text in text_for_predict]

In [317]:
# predict 
df_predict = vect_tfidf.transform(text_for_predict)

In [318]:
y_pred_proba = clf_logreg.predict_proba(df_predict)

In [329]:
max_10_index_logreg = (-y_pred_proba[:, 1]).argsort()[:10]

In [331]:
max_10_index_logreg

array([10868, 66922, 46397, 28566,  8084, 69075, 52710, 58455, 61532,
       64544])

In [333]:
max_10_prob_logreg = -np.sort((-y_pred_proba[:, 1]))[:10]

# 2 подход

In [387]:
vect_count = CountVectorizer(
    ngram_range=(1,2),
    min_df = 5,
    max_df = 0.8,
    max_features = 3000,
    token_pattern=r'(?u)\b\w+\b'
)

In [388]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], target, train_size=0.2, stratify=target)

In [389]:
X_train = vect_count.fit_transform(X_train)
X_test = vect_count.transform(X_test)

In [396]:
clf_knn = KNeighborsClassifier(n_neighbors=10, n_jobs=-1, metric='cosine', algorithm='brute')

In [397]:
clf_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_jobs=-1,
                     n_neighbors=10)

In [398]:
y_pred = clf_knn.predict(X_test)

In [399]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.75      0.83      0.79      7669
         1.0       0.58      0.46      0.51      3861

    accuracy                           0.71     11530
   macro avg       0.67      0.65      0.65     11530
weighted avg       0.70      0.71      0.70     11530



In [400]:
df_predict = vect_count.transform(text_for_predict)

In [401]:
y_pred_proba = clf_knn.predict_proba(df_predict)

In [402]:
max_10_index_knn = (-y_pred_proba[:, 1]).argsort()[:10]

In [403]:
max_10_prob_knn = -np.sort((-y_pred_proba[:, 1]))[:10]

# Сравниваем

## logreg

In [405]:
for i in range(len(max_10_index_logreg)):
    print(f'вероятность токсичности {max_10_prob_logreg[i]}')
    print('Текст')
    print(text_for_predict[max_10_index_logreg[i]])

вероятность токсичности 0.9945216223967746
Текст
 хохол ты обосраться он твой соотечественник.

вероятность токсичности 0.9936697394200873
Текст
 уиииииииииииия ряя ты тупой ряять уиииия не согласный с пидорашкой-ты хохол уиииииииии!стеклома накатить свинья руснявая.

вероятность токсичности 0.990700437455225
Текст
 поздравлять ты хохол

вероятность токсичности 0.99043207575544
Текст
 нахуй ты он задаешь?

вероятность токсичности 0.9899499723024717
Текст
 ты запятая пропустилить ты просто тупой

вероятность токсичности 0.989385172985289
Текст
 ахи блядь андрей ты долбоести нахуй ты кроссовок фоткаешь)

вероятность токсичности 0.9869712649913053
Текст
 нахуй ты свой батя читаешь?

вероятность токсичности 0.9864229512022662
Текст
 игор ты сука ты чмо ты пидор ты гной блядь ты нахуй варга убить сука как я теперь новый альбом ждать чмо всратое?

вероятность токсичности 0.9858708823481218
Текст
 таракан ты или ты просто тупой ебанат?

вероятность токсичности 0.9844633036686685
Текст
 хохол 

## KNN

In [407]:
for i in range(len(max_10_index_knn)):
    print(f'вероятность токсичности {max_10_prob_knn[i]}')
    print('Текст')
    print(text_for_predict[max_10_index_knn[i]])

вероятность токсичности 1.0
Текст
  я смотреть на ты ты глядеть на я искра буря безумие.

вероятность токсичности 1.0
Текст
 я ты  пидор первый посадить на вилы с такой идеями.

вероятность токсичности 1.0
Текст
   да нахуй ты нужный лол.

вероятность токсичности 1.0
Текст
 писать сюда быть разбираться из данные профиль видно что ты уже просто всё заебало в остальной у ты всё вроде нормально психически здоров.

вероятность токсичности 1.0
Текст
 че ты тут забыть хиккан  здесь всё нормальный социоблядь сидят

вероятность токсичности 1.0
Текст
 просто ты натурал но.

вероятность токсичности 1.0
Текст
 просто ты трусливый мразь

вероятность токсичности 1.0
Текст
 да флаг ты в рука блядь хулить ты такой уёбок то прийти выпендриться какой ты умный  постить книжка не быть делать это умный человек не быть он спрашивать что читать а что нет потому что умный сам посомтрит и по крупица собрать зерно истина из каждый книжка а ты лишь позер.

вероятность токсичности 1.0
Текст
 нет ты адрес дать чм

Тексты не совпаают, но все очень токсчиное. Вероятность 1 в версии с KNN т.к. он не умеет в вероятность.